# 🫀 Clinical-Grade Multimodal ECG Training Pipeline  
<span style="color:red">by Ridwan Oladipo, MD | Medical AI Specialist</span>  

Production-ready training pipeline for **12-lead ECG classification**, implementing a **ResNet-1D + tabular fusion network** with:  

- **ResNet-1D signal branch** → temporal P–QRS–T wave & rhythm morphology modeling  
- **Clinical metadata branch** → HR/HRV + age/sex + device harmonization  
- **Late fusion** → integrated ECG + tabular decision space
- **Binary cross-entropy loss** for multilabel setting  
- **Recall-optimized callbacks** → early stopping & checkpointing to maximize **myocardial infarction sensitivity**  
- **Reproducible training** with fixed seeds & official PTB-XL stratified folds  (preventing patient leakage)

🚀 Trains on **~17k+ ECGs** with structured logging & TensorBoard monitoring.  
>⚕️ **Clinically-aligned optimization** — tuning for **sensitivity and NPV in myocardial infarction detection**, the metrics that matter most in cardiology.